# Semesterarbeit Teil 2

Bei diesem Teil der Semesterarbeit ist das Ziel eine eigene Python-Klasse zu erstellen. Die Klasse sollte in der Lage sein, die weit verbreiteten NLP- Ansätze der term frequency und inverse document frequency umzusetzen. Die txt-Dokumente befinden sich in der selben directory wie das Py-file und das Jupyternotebook. Bei einem anderen Speicherort, müsste man den Dateipfad ausschreiben. 

Nachfolgend die nötigen Definitionen der TF und IDF - Methoden:

Die inverse document frequency : $idf(wort)$ ist der Logarithmus des Verhältnisses der Länge des Korpus (Anzahl Dictionaries) zur Anzahl derer Dokumente, in welchen das gesuchte Wort enthalten ist.

$\text{idf}(\text{wort}) = \log_{10}\left(\frac{\text{len(korpus)}}{\text{df   +1}}\right)$

df = Anzahl Dokumente aus dem Korpus welche Wort enthalten (1 Vorkommen genügt). Das Ziel der IDF ist es seltene Wörter in dem Datensatz hochzuskalieren. Ein Füllwort, welches oft vorkommt hat eine hohe TF ist aber nicht wirklich wichtig. Ein Wort, dass in allen Datensätzen vorkommt wird einen IDF von 0 bekommen, da $ log(N/N) = 0$ ergibt. Je seltener das Wort, desto grösser wird der Bruch und somit IDF.

Für sehr grosse Texte müssen wir den logarithmus zur Basis 10 nehmen, damit der idf-score nicht durch die Decke schiesst. Für Texte in denen ein Wort nicht vorkommt wird DF 0 sein. Da wir nicht durch 0 dividieren können addieren wir 1 zum Nenner.


Term Frequency misst die relative Häufigkeit eines Wortes innerhalb eines vorgegebenen Dokuments. Ein Wort welches oft vorkommt, erhält folgerichtig auch einen hohen Wert. Füllwörter und Konjunktionen werden vorderhand oft aus dem Text-string herausgerechnet, da diese immer sehr häufig vorkommen. Der Wertebereich liegt zwischen 0 und 1:

$\text{tf(wort, dokument)} = \frac{\text{Anzahl Vorkommen von wort im Dokument}}{\text{Gesamtzahl aller Wörter im Dokument}}$


Schlussendlich berechnet man den TF-IDF Score 
 tf(wort, dokument) * idf(wort, korpus)


In [1]:
import math
import pickle

In [2]:
class Corpus :
    """
    Die Klasse Corpus liest und analysiert Dokumente, welche wir mit der Hilf
    klasse Document zuvor eingelesen haben. Zuerst wird aus dem txt.file die
    Wörter herausgefiltert in einen Dictionary mit keys = wort und values = Anzahl Vorkommen des Wortes
    im key. Die Methoden tf und idf beziehen sich dann auf den Dictionary
    
    """
    
    def __init__(self) : #initialise
        self.documents = [] 
    
             
    def read_document(self, document_name, path) :
        
        text =[]#Hilfsvariablen brauchen Zuordnung
        with open (path) as datei: #Hilfsvariablen brauchen Zuordnung
            for line in datei:
                text += line.split()
        
        result = {}
        for w in text:
            result[w] = text.count(w)
                    
        self.documents.append(Document(document_name, path, result))
        #return result
    
    def list_documents_names(self) : 
         
        result = []
        for doc in self.documents: 
            result.append(doc.name)
        return result
    
     
    
    def print_document(self, document) :
        "Ausdrucken des Inhalts des angegebenen Inhalts" 
        dok = self.find_document(document)
        if dok != None :
            print(dok.inhalt)         

    
    def remove_document(self, document_name) :
        "Entfernt ein Dokument welches sich fälchlischerweise eingeschlichen hat"
        for doc in self.documents:
            if document_name == doc.name :
                self.documents.remove(doc)
        
    
    def tf(self, term, document_name) :
        "Falls term in "
        
        document = self.find_document(document_name)
        
        if term in document.word_count:
            x = document.word_count[term]
            return x / max(document.word_count.values())        
        else:
            return 0
        
    
        
    def idf(self, term) :
        "mit dem Zähler zählen wir alle Dokumente in denen term vorkommt"
        
        zaehler = 0
        for doc in self.documents:
            if term in doc.word_count:
                zaehler += 1
               
        return math.log10 (len(corp.documents) /(zaehler + 1))
        
    def tf_idf(self, term, document_name) :
        
        return self.tf(term, document_name) * self.idf(term)
    
    def tf_idf_all(self, document_name):
        "Dies würde einen Dictionary mit den tf--idf scores aller Wörter generieren"
        result ={}
        for term in self.find_document(document_name).word_count:
            result[term] =  self.tf_idf(term, document_name)
        return result
            
        
    def find_document(self, document_name) : #Hilfsmethode
        for doc in self.documents:
            if ( doc.name == document_name):
                return doc
        print ("Kein Dokument vorhanden")
     

In [4]:
corp = Corpus()

In [5]:
corp.read_document('doc1','input1.txt')
corp.read_document('doc2','input2.txt')
corp.read_document('doc3','input3.txt')
corp.read_document('doc4','input4.txt')
corp.read_document('doc5','input5.txt')
corp.documents

[Document doc1 has count {'Helmut': 6, 'Kohl': 3, 'Insel': 3, 'Bruder': 3, 'Fritz': 1, 'Hilfe': 1},
 Document doc2 has count {'Nikosia': 1, 'Kohl': 3, 'Helmut': 5, 'Insel': 5, 'Bruder': 3, 'Fritz': 1, 'Muster': 1, 'Hilfe': 1},
 Document doc3 has count {'Helmut': 4, 'Kohl': 1, 'Insel': 5, 'Nikosia': 1, 'Bruder': 4, 'Fritz': 1},
 Document doc4 has count {'Helmut': 4, 'Kohl': 2, 'Insel': 8, 'Bruder': 5, 'Fritz': 2, 'Muster': 1},
 Document doc5 has count {'Insel': 3, 'Helmut': 3, 'Hilfe': 2, 'Bruder': 4, 'Fritz': 2, 'Muster': 1}]

In [6]:
corp.remove_document('doc2')
corp.documents

[Document doc1 has count {'Helmut': 6, 'Kohl': 3, 'Insel': 3, 'Bruder': 3, 'Fritz': 1, 'Hilfe': 1},
 Document doc3 has count {'Helmut': 4, 'Kohl': 1, 'Insel': 5, 'Nikosia': 1, 'Bruder': 4, 'Fritz': 1},
 Document doc4 has count {'Helmut': 4, 'Kohl': 2, 'Insel': 8, 'Bruder': 5, 'Fritz': 2, 'Muster': 1},
 Document doc5 has count {'Insel': 3, 'Helmut': 3, 'Hilfe': 2, 'Bruder': 4, 'Fritz': 2, 'Muster': 1}]

Die Methode remove_document loopt durch die Dokumente welche sich in dem Klassenattribut self.dokumente befinden und macht einen Abgleich mit dem Namen einer Hilfsvariable für Dokumente. 

In [13]:
print(len(corp.documents))

4


Die built-in len() Funktion ist nützlich für die idf- Methode, da wir sie verwenden um den Zähler zu bestimmen.

In [14]:
corp.list_documents_names()

['doc1', 'doc3', 'doc4', 'doc5']

Diese Methode populiert eine leere Liste mit den Ergebnissen einer for-Schleife welche durch alle Dokumente innerhalb des Klassenattributs Dokumente geht.

In [15]:
corp.

SyntaxError: invalid syntax (<ipython-input-15-0efe20afac88>, line 1)

In [3]:
class Document :
    """
        Hilfsklasse für ein Dokument. 
        Eine Instanz dieser Klasse enthält
        Name und Pfad des Dokuments, den Inhalt des Dokuments als String
        und ein Dictionary mit Word-Counts.
    """
    def __init__(self, name, path, word_count) :
        self.name = name
        self.path = path
        #self.content = content # Inhalts-text als string
        self.word_count = word_count
        
    def __repr__(self) : #es zeigt eine vorschau des ausdrucks, wie soll das dokument beschrieben 
    #builtin-klassen haben diese Vorschau bereits- siehe z.B die Klasse LISTE!
    
        return "Document {} has count {}".format(self.name,self.word_count)
    
    def __str__(self):
        return "Der Dokumentname lautet:{} mit {} Wörtern".format(self.name,self.word_count)
    # gibt einen String aus- 


In [16]:
print(doc1)

NameError: name 'doc1' is not defined

In [17]:
corp.tf('Nikosia','doc1')

0

Es macht Sinn, dass das Wort Nikosia eine tf von 0 in doc1 erhält, da es kein einzige Mal vorkommt. 

In [24]:
corp.idf('Nikosia')

0.3010299956639812

Die Funktion idf ergibt für das Wort Helmut einen Wert von -0.09691001. Dies deutet auf ein hohes Vorkommen des Wortes Helmut hin und tatsächlich kommt dieses Wort in allen Dokumenten vor. Wir behelfen uns der Hilfsmethode find_document um eine hilfsvariable Dokument mit dem Attribut word_count der Dokuments-klasse zu verknüpfen. Wir dividieren das Vorkommen von wort term durch das Maximum an Wörtern in dem Dokument. Wenn term nicht vorkomm geben wir 0 aus. 

In [19]:
x = math.log10(4/5)
x #Nachweis das idf richtig rechnet

-0.09691001300805639

In [22]:
corp.tf_idf('Kohl','doc1')

0.0

In [7]:
corp.tf_idf_all('doc2')

Kein Dokument vorhanden


AttributeError: 'NoneType' object has no attribute 'word_count'

In [ ]:
pickle

In [10]:
file_name='corp.pkl'

f = open(file_name,'wb')
pickle.dump(corp,f)
f.close()